In [1]:
import cryo
import polars as pl
import json

## Constants

In [2]:
CREATION_MARKET_TOPIC = "0xac4b2400f169220b0c0afdde7a0b32e775ba727ea1cb30b35f935cdaab8683ac"
INTEREST_TOPIC = "0x9d9bd501d0657d7dfe415f779a620a62b78bc508ddc0891fbbd8b7ac0f8fce87"
BORROW_TOPIC = "0x570954540bed6b1304a87dfe815a5eda4a648f7097a16240dcd85c9b5fd42a43"
REPAY_TOPIC = "0x52acb05cebbd3cd39715469f22afbf5a17496295ef3bc9bb5944056c63ccaa09"
SUPPLY_COLLATERAL_TOPIC = "0xa3b9472a1399e17e123f3c2e6586c23e504184d504de59cdaa2b375e880c6184"
WITHDRAW_COLLATERAL_TOPIC = "0xe80ebd7cc9223d7382aab2e0d1d6155c65651f83d53c8b9b06901d167e321142"
SUPPLY_TOPIC = "0xedf8870433c83823eb071d3df1caa8d008f12f6440918c20d75a3602cda30fe0"
WITHDRAW_TOPIC = "0xa56fc0ad5702ec05ce63666221f796fb62437c32db1aa1aa075fc6484cf58fbf"
LIQUIDATION_TOPIC = "0xa4946ede45d0c6f06a0f5ce92c9ad3b4751452d2fe0e25010783bcab57a67e41"

# Id is an alias for bytes32
# MarketParams is a struct with the following fields:
# - address loanToken
# - address collateralToken
# - address oracle
# - address irm
CREATION_MARKET_SIG = "event CreateMarket(Id indexed id, MarketParams marketParams)".replace('Id', "bytes32")\
    .replace('MarketParams marketParams', 'address loanToken, address collateralToken, address oracle, address irm, uint256 lltv)')
SUPPLY_SIG = "event Supply(Id indexed id, address indexed caller, address indexed onBehalf, uint256 assets, uint256 shares)".replace('Id', "bytes32")
WITHDRAW_SIG = "event Withdraw(Id indexed id,address caller,address indexed onBehalf,address indexed receiver,uint256 assets,uint256 shares);".replace('Id', "bytes32")
BORROW_SIG = "event Borrow(Id indexed id, address caller, address indexed onBehalf, address indexed receiver, uint256 assets, uint256 shares)".replace('Id', "bytes32")
REPAY_SIG = "event Repay(Id indexed id, address indexed caller, address indexed onBehalf, uint256 assets, uint256 shares)".replace('Id', "bytes32")
SUPPLY_COLLATERAL_SIG = "event SupplyCollateral(Id indexed id, address indexed caller, address indexed onBehalf, uint256 assets)".replace('Id', "bytes32")
WITHDRAW_COLLATERAL_SIG = "event WithdrawCollateral(Id indexed id, address caller, address indexed onBehalf, address indexed receiver, uint256 assets)".replace('Id', "bytes32")
LIQUIDATION_SIG = "event Liquidate(Id indexed id,address indexed caller,address indexed borrower,uint256 repaidAssets,uint256 repaidShares,uint256 seizedAssets,uint256 badDebtAssets,uint256 badDebtShares);".replace('Id', "bytes32")
INTEREST_SIG = "event AccrueInterest(Id indexed id, uint256 prevBorrowRate, uint256 interest, uint256 feeShares);".replace('Id', "bytes32")

BLUE_CONTRACT_ADDRESS = "0xBBBBBbbBBb9cC5e90e3b3Af64bdAF62C37EEFFCb"


## Utils

In [3]:


def convert_to_type(x, type, name="", data_offset = 0):
    ''' Convert raw data to a specific type
    x: the raw data (bytes)
    type: the type of the data
    name: the name of the data
    data_offset: the offset of the data in the raw data'''
    
    if type == 'string':
        offset = int(x[data_offset+12: data_offset + 32].hex(), 16)
        length = int(x[offset:offset + 32].hex(), 16)
        return x[offset + 32: offset + 32 + length].decode('utf-8')
    elif '[]' in type:
        # TODO
        return '0x'
    elif len(type) > 4 and type[:4] == 'uint':
        return float(int(x[data_offset+22: data_offset+32].hex(), 16))
    elif type == 'address':
        return '0x' + x[12+data_offset:32+data_offset].hex()
    elif type == 'bool':
        return int(x[data_offset: 32 + data_offset], 16) > 0
    elif len(type) > 5 and type[:5] == 'bytes':
        bytes_left = 32 - int(type[5:])
        return '0x' + x[data_offset+bytes_left: data_offset+32].hex()
    else:
        return x
    
def parse_raw_events(raw_events: pl.DataFrame, signature: str) -> pl.DataFrame:
    '''
    Parse raw events from a DataFrame, this dataframe should contain the following columns:
    - topic0 to topic3: the topics of the event
    - data: the data of the event
    The signature is used to determine the type of the event, it should look like this:
    "EventName(uint256 indexed arg1, uint256 indexed arg2, T arg3, ...)"

    The function works with basic types (string, uint, address, bool, bytes) but not yet (TODO) with arrays or structs.
    '''
    
    # Parse the signature
    first_parenthesis = signature.find('(')
    last_parenthesis = signature.rfind(')')

    print(f'Parsing Event name: {signature[:first_parenthesis]}')

    types_str = signature[first_parenthesis + 1:last_parenthesis]
    types = types_str.split(',')
    indexed_types, data_types = [], []

    for t in types:
        parts = t.strip().split(' ')
        if 'indexed' in parts:
            indexed_types.append((parts[0], parts[-1]))
        else:
            data_types.append((parts[0], parts[-1]))

    dfs = []
    for index, (type, name) in enumerate(data_types):
        data_events_cur_df = raw_events.select([
            pl.col("data").map_elements(lambda x: convert_to_type(x, type, name, 32*index)).alias(name)
        ] + [pl.col("block_number"), pl.col("transaction_index", "log_index")])
        dfs.append(data_events_cur_df)

    data_events_df = raw_events.select([
        pl.col("data").map_elements(lambda x: convert_to_type(x, type, name, 32*index)).alias(name)
        for index, (type, name) in enumerate(data_types)
    ] + [pl.col("block_number"), pl.col("transaction_index", "log_index")])
    indexed_events_df = raw_events.select([
        pl.col(f"topic{1+index}").map_elements(lambda x: convert_to_type(x, type)).alias(name)
        for index, (type, name) in enumerate(indexed_types)
    ] + [pl.col("block_number"), pl.col("transaction_index", "log_index")])

    print(f'Indexed types: {indexed_types}')
    print(f'Data types: {data_types}')
    

    res_df = indexed_events_df
    for df in dfs:
        res_df = res_df.join(df, on=["block_number", "transaction_index", "log_index"], how="inner")

    return res_df

# Blue market creation



In [4]:

# Collect the raw events with specific cryo params
params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [CREATION_MARKET_TOPIC],
}

# Collect the raw events with cryo
blue_market_creations_raw = cryo.collect(
    **params
)

blue_market_creations_raw.head(5)

block_number,transaction_index,log_index,transaction_hash,address,topic0,topic1,topic2,topic3,data,chain_id
u32,u32,u32,binary,binary,binary,binary,binary,binary,binary,u64
18919623,73,135,"b""\x16\x1c\x8d\xb8X\x86\x1e\xa5\x11\x9dhl\xde#\xe3\xda\xbb\x88\xd3\xd2\xd2\xa9\xe1\x172\xca=\x96`\xfc\x97\xf6""","b""\xbb\xbb\xbb\xbb\xbb\x9c\xc5\xe9\x0e;:\xf6K\xda\xf6,7\xee\xff\xcb""","b""\xacK$\x00\xf1i""\x0b\x0c\x0a\xfd\xdez\x0b2\xe7u\xbar~\xa1\xcb0\xb3_\x93\\xda\xab\x86\x83\xac""","b""\xc5Mz\xcf\x14\xde)\xe0\xe5R|\xab\xd7\xa5vPhp4jx\xa1\x1agb\xe2\xcc\xa6c""\xecA""",null,null,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0*\xaa9\xb2#\xfe\x8d\x0a\x0e\O'\xea\xd9\x08<ul\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7f9\xc5\x81\xf5\x95\xb5<\\xb1\x9b\xd0\xb3\xf8\xdal\x93^,\xa0""...",1
18919858,95,141,"b""\xea\x1co\xae\xa5\x17)\xc6\x91\x92\x98\xd6\x0e\x99\xee\x20\xd1\xb4^<\xec\x993\xeac\x8c\xc1\xa6\x96$\xb9\xdf""","b""\xbb\xbb\xbb\xbb\xbb\x9c\xc5\xe9\x0e;:\xf6K\xda\xf6,7\xee\xff\xcb""","b""\xacK$\x00\xf1i""\x0b\x0c\x0a\xfd\xdez\x0b2\xe7u\xbar~\xa1\xcb0\xb3_\x93\\xda\xab\x86\x83\xac""","b""T\xef\xde\xe0\x8e'.\x92\x904\xa8\xf2o|\xa3K\x1e\xbe6K'S\x91\x16\x9b(\xc6\xd7\xdb$\xdb\xc8""",null,null,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""...",1
18919860,80,255,"b""\xbe\x0a\x1a\x90x\xef\xed\xa0\xa1\xbb\xf0\x90Lr\x03\xa0\x90Zc\xb1kc\xe1\x05'J*\x19\x91_w\x9d""","b""\xbb\xbb\xbb\xbb\xbb\x9c\xc5\xe9\x0e;:\xf6K\xda\xf6,7\xee\xff\xcb""","b""\xacK$\x00\xf1i""\x0b\x0c\x0a\xfd\xdez\x0b2\xe7u\xbar~\xa1\xcb0\xb3_\x93\\xda\xab\x86\x83\xac""","b""X\xe2\x12\x06\x06E\xd1\x8e\xabm\x9b*\xf3\xd5o\xbc\x90j\x92\xffVg8_aof,p7""\x84""",null,null,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0*\xaa9\xb2#\xfe\x8d\x0a\x0e\O'\xea\xd9\x08<ul\xc2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""...",1
18925864,126,392,"b""%L]\xd1x6\xbc\x80\xa3\xe4\xd6\xfe\xfe\xe4|\xae{N?\xe6\x90D\x19?\xab##\x94\x0e\xdd8\xa0""","b""\xbb\xbb\xbb\xbb\xbb\x9c\xc5\xe9\x0e;:\xf6K\xda\xf6,7\xee\xff\xcb""","b""\xacK$\x00\xf1i""\x0b\x0c\x0a\xfd\xdez\x0b2\xe7u\xbar~\xa1\xcb0\xb3_\x93\\xda\xab\x86\x83\xac""","b""\x08SX\x10\xfe\xad\xf5-a\xa1\x82\xdcz\x12\xbax\xf9\xb3\x8a\x82\x9e\xe4w\xe0\xe1\xd4\x88\xd4\xe2b\xbc\x0a""",null,null,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xab\xc5a\x86\xc2\xdfc\xc38\xe0L\x9c\x9b\x98\x14\xe2y\x83\xa5\xa9""...",1
18925910,94,214,"b""\xf7f\xfa\x95`\x04\xbc\xea\x9b\x0eW\xc4V/\xf9T\xed\xfd0f\xcd\x9f!\xe2\x8f\xba\xba\xe4\x01\xb7\x8e\xde""","b""\xbb\xbb\xbb\xbb\xbb\x9c\xc5\xe9\x0e;:\xf6K\xda\xf6,7\xee\xff\xcb""","b""\xacK$\x00\xf1i""\x0b\x0c\x0a\xfd\xdez\x0b2\xe7u\xbar~\xa1\xcb0\xb3_\x93\\xda\xab\x86\x83\xac""","b""\xb3#I_~AH\xbeVC\xa4\xeaJ\x82!\xee\xf1c\xe4\xbc\xcf\xde\xdc*oF\x96\xba\xac\xbc\x86\xcc""",null,null,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x7f9\xc5\x81\xf5\x95\xb5<\\xb1\x9b\xd0\xb3\xf8\xdal\x93^,\xa0""...",1


In [5]:
# Parse the raw events to a DataFrame with a specific event signature

blue_market_creations = parse_raw_events(blue_market_creations_raw, 
"event CreateMarket(bytes32 indexed id, address loanToken, address collateralToken, address oracle, address irm, uint256 lltv)")

blue_market_creations.head(5)

Parsing Event name: event CreateMarket
Indexed types: [('bytes32', 'id')]
Data types: [('address', 'loanToken'), ('address', 'collateralToken'), ('address', 'oracle'), ('address', 'irm'), ('uint256', 'lltv')]


id,block_number,transaction_index,log_index,loanToken,collateralToken,oracle,irm,lltv
str,u32,u32,u32,str,str,str,str,f64
"""0xc54d7acf14de…",18919623,73,135,"""0xc02aaa39b223…","""0x7f39c581f595…","""0x2a01eb949609…","""0x870ac11d48b1…",9.4500e17
"""0x54efdee08e27…",18919858,95,141,"""0xa0b86991c621…","""0x000000000000…","""0x000000000000…","""0x000000000000…",0.0
"""0x58e212060645…",18919860,80,255,"""0xc02aaa39b223…","""0x000000000000…","""0x000000000000…","""0x000000000000…",0.0
"""0x08535810fead…",18925864,126,392,"""0xa0b86991c621…","""0xabc56186c2df…","""0x6e8f5b2df218…","""0x870ac11d48b1…",9.6500e17
"""0xb323495f7e41…",18925910,94,214,"""0xa0b86991c621…","""0x7f39c581f595…","""0x48f7e36eb6b8…","""0x870ac11d48b1…",8.6000e17


# Interactions

## Supply

In [6]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [SUPPLY_TOPIC],
}
blue_supply_raw = cryo.collect(
    **params
)

blue_supply = parse_raw_events(blue_supply_raw, SUPPLY_SIG)

blue_supply.head(5)

Parsing Event name: event Supply
Indexed types: [('bytes32', 'id'), ('address', 'caller'), ('address', 'onBehalf')]
Data types: [('uint256', 'assets'), ('uint256', 'shares')]


id,caller,onBehalf,block_number,transaction_index,log_index,assets,shares
str,str,str,u32,u32,u32,f64,f64
"""0x4a8221eef163…","""0x9106cbf2c882…","""0x9106cbf2c882…",18926166,104,208,1e8,1.0000e14
"""0x4a8221eef163…","""0xbeef01735c13…","""0xbeef01735c13…",18940531,93,259,1.0000e9,9.9999e14
"""0xd7a576506870…","""0x38989bba00bd…","""0x38989bba00bd…",18941243,38,105,5.0000e16,5.0000e22
"""0x4a8221eef163…","""0xbeef01735c13…","""0xbeef01735c13…",18941650,87,173,1.0000e10,9.9999e15
"""0x4a8221eef163…","""0xbeef01735c13…","""0xbeef01735c13…",18942211,74,97,2e8,2.0000e14


## Withdraw

In [7]:
params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [WITHDRAW_TOPIC],
}

blue_withdraw_raw = cryo.collect(
    **params
)

blue_withdraw = parse_raw_events(blue_withdraw_raw, WITHDRAW_SIG)

blue_withdraw.head(5)

Parsing Event name: event Withdraw
Indexed types: [('bytes32', 'id'), ('address', 'onBehalf'), ('address', 'receiver')]
Data types: [('address', 'caller'), ('uint256', 'assets'), ('uint256', 'shares')]


id,onBehalf,receiver,block_number,transaction_index,log_index,caller,assets,shares
str,str,str,u32,u32,u32,str,f64,f64
"""0xd7a576506870…","""0x38989bba00bd…","""0x38989bba00bd…",18941407,66,104,"""0x38989bba00bd…",1.2500e16,1.2500e22
"""0x4a8221eef163…","""0xbeef01735c13…","""0xbeef01735c13…",18941650,87,180,"""0xbeef01735c13…",1.0000e9,9.9999e14
"""0x4a8221eef163…","""0xbeef01735c13…","""0xbeef01735c13…",18963222,37,88,"""0xbeef01735c13…",1.0,999992.0
"""0xd7a576506870…","""0x38989bba00bd…","""0x38989bba00bd…",18965496,93,234,"""0x38989bba00bd…",1.0010e15,1.0010e21
"""0xd7a576506870…","""0x38989bba00bd…","""0x38989bba00bd…",18969656,76,146,"""0x38989bba00bd…",2.3000e15,2.2999e21


## Supply collateral 

In [8]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [SUPPLY_COLLATERAL_TOPIC]
}

blue_supply_collateral_raw = cryo.collect(
    **params
)

blue_supply_collateral = parse_raw_events(blue_supply_collateral_raw, SUPPLY_COLLATERAL_SIG)

blue_supply_collateral.head(5)

Parsing Event name: event SupplyCollateral
Indexed types: [('bytes32', 'id'), ('address', 'caller'), ('address', 'onBehalf')]
Data types: [('uint256', 'assets')]


id,caller,onBehalf,block_number,transaction_index,log_index,assets
str,str,str,u32,u32,u32,f64
"""0x4a8221eef163…","""0x9106cbf2c882…","""0x9106cbf2c882…",18926070,88,135,5.0000e16
"""0xd7a576506870…","""0xa7995f71aa11…","""0x4563364bd619…",18942176,109,204,4.0000e16
"""0x4a8221eef163…","""0xa7995f71aa11…","""0x4563364bd619…",18942176,109,207,4.6777e16
"""0xd7a576506870…","""0xa7995f71aa11…","""0x6e632701fd42…",18943134,132,344,1.0000e14
"""0x4a8221eef163…","""0xa7995f71aa11…","""0x2b5469940fa5…",18943214,93,252,1.0000e16


## Withdraw collateral

In [9]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [WITHDRAW_COLLATERAL_TOPIC]
}

blue_withdraw_collateral_raw = cryo.collect(
    **params
)

blue_withdraw_collateral = parse_raw_events(blue_withdraw_collateral_raw, WITHDRAW_COLLATERAL_SIG)

blue_withdraw_collateral.head(5)

Parsing Event name: event WithdrawCollateral
Indexed types: [('bytes32', 'id'), ('address', 'onBehalf'), ('address', 'receiver')]
Data types: [('address', 'caller'), ('uint256', 'assets')]


id,onBehalf,receiver,block_number,transaction_index,log_index,caller,assets
str,str,str,u32,u32,u32,str,f64
"""0xd7a576506870…","""0x4563364bd619…","""0xa7995f71aa11…",18942670,70,189,"""0xa7995f71aa11…",1.0000e16
"""0x4a8221eef163…","""0x2b5469940fa5…","""0x2b5469940fa5…",18965474,31,111,"""0xa7995f71aa11…",9.9997e15
"""0xd7a576506870…","""0x6e632701fd42…","""0xa7995f71aa11…",18975191,117,190,"""0xa7995f71aa11…",1.0000e14
"""0x4a8221eef163…","""0x9cbf099ff424…","""0x9cbf099ff424…",18977263,122,221,"""0xa7995f71aa11…",1.0000e19
"""0x695d96d685fa…","""0x1f381f47b9c7…","""0x1f381f47b9c7…",18982531,160,278,"""0x1f381f47b9c7…",2.0000e17


## Borrow

In [10]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [BORROW_TOPIC]
}

blue_borrow_raw = cryo.collect(
    **params
)

blue_borrow = parse_raw_events(blue_borrow_raw, BORROW_SIG)

blue_borrow.head(5)


Parsing Event name: event Borrow
Indexed types: [('bytes32', 'id'), ('address', 'onBehalf'), ('address', 'receiver')]
Data types: [('address', 'caller'), ('uint256', 'assets'), ('uint256', 'shares')]


id,onBehalf,receiver,block_number,transaction_index,log_index,caller,assets,shares
str,str,str,u32,u32,u32,str,f64,f64
"""0x4a8221eef163…","""0x9106cbf2c882…","""0x9106cbf2c882…",18926262,121,190,"""0x9106cbf2c882…",1e7,1.0000e13
"""0xd7a576506870…","""0x4563364bd619…","""0xa7995f71aa11…",18942176,109,212,"""0xa7995f71aa11…",1.0000e16,1.0000e22
"""0x4a8221eef163…","""0x4563364bd619…","""0x4563364bd619…",18942176,109,216,"""0xa7995f71aa11…",8.8021751e7,8.8016e13
"""0xd7a576506870…","""0x6e632701fd42…","""0x6e632701fd42…",18943134,132,349,"""0xa7995f71aa11…",1.0774e14,1.0774e20
"""0x4a8221eef163…","""0x2b5469940fa5…","""0x2b5469940fa5…",18943214,93,257,"""0xa7995f71aa11…",3.826194e6,3.8259e12


## Repay

In [11]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [REPAY_TOPIC]
}

blue_repay_raw = cryo.collect(
    **params
)

blue_repay = parse_raw_events(blue_repay_raw, REPAY_SIG)

blue_repay.head(5)

Parsing Event name: event Repay
Indexed types: [('bytes32', 'id'), ('address', 'caller'), ('address', 'onBehalf')]
Data types: [('uint256', 'assets'), ('uint256', 'shares')]


id,caller,onBehalf,block_number,transaction_index,log_index,assets,shares
str,str,str,u32,u32,u32,f64,f64
"""0x4a8221eef163…","""0xa7995f71aa11…","""0x4563364bd619…",18942670,70,185,4e7,3.9997e13
"""0x4a8221eef163…","""0xa7995f71aa11…","""0x2b5469940fa5…",18949974,81,160,100000.0,9.9991e10
"""0x4a8221eef163…","""0xa7995f71aa11…","""0x2b5469940fa5…",18965474,31,109,3.726e6,3.7254e12
"""0xd7a576506870…","""0xa7995f71aa11…","""0x6e632701fd42…",18975191,117,179,1.0775e14,1.0774e20
"""0x4a8221eef163…","""0x9106cbf2c882…","""0x9106cbf2c882…",18982559,54,77,9.5115e9,9.5088e15


## Liquidate 

In [12]:
params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [LIQUIDATION_TOPIC]
}

blue_liquidation_raw = cryo.collect(
    **params
)

blue_liquidation = parse_raw_events(blue_liquidation_raw, LIQUIDATION_SIG)

blue_liquidation.head(5)

Parsing Event name: event Liquidate
Indexed types: [('bytes32', 'id'), ('address', 'caller'), ('address', 'borrower')]
Data types: [('uint256', 'repaidAssets'), ('uint256', 'repaidShares'), ('uint256', 'seizedAssets'), ('uint256', 'badDebtAssets'), ('uint256', 'badDebtShares')]


id,caller,borrower,block_number,transaction_index,log_index,repaidAssets,repaidShares,seizedAssets,badDebtAssets,badDebtShares
str,str,str,u32,u32,u32,f64,f64,f64,f64,f64
"""0x4a8221eef163…","""0x5343d15decb9…","""0x4563364bd619…",18976844,127,34,1.06555546e8,1.0653e14,4.0643e16,0.0,0.0
"""0xf6ec69ce473d…","""0x5343d15decb9…","""0xa4459ccbd068…",19130360,56,155,10425.0,1.0369e10,26.0,0.0,0.0
"""0xcaafcf8cd4b9…","""0xd85e92f58a1f…","""0x255c7705e8bb…",19206767,38,41,1.02435505e8,1.0000e14,1.0692e20,0.0,0.0
"""0xe4b154e16c97…","""0x5343d15decb9…","""0x04c0fd5032c1…",19216001,39,42,8.3371085e7,8.3355e13,174417.0,0.0,0.0
"""0x86b2f3f9d78e…","""0x8ce45e650ab1…","""0x17fd03969575…",19250804,2,19,1.6490e18,4.2956e23,1.6708e18,0.0,0.0


## Interest

In [13]:

params = {
    "datatype": "events",
    "blocks": ["18.9M:"],
    "inner_request_size": 10000,
    "contract": [BLUE_CONTRACT_ADDRESS],
    "topic0": [INTEREST_TOPIC]
}

blue_interest_raw = cryo.collect(
    **params
)

blue_interest = parse_raw_events(blue_interest_raw, INTEREST_SIG)

blue_interest.head(5)

Parsing Event name: event AccrueInterest
Indexed types: [('bytes32', 'id')]
Data types: [('uint256', 'prevBorrowRate'), ('uint256', 'interest'), ('uint256', 'feeShares')]


id,block_number,transaction_index,log_index,prevBorrowRate,interest,feeShares
str,u32,u32,u32,f64,f64,f64
"""0xb323495f7e41…",18926166,104,207,3.16327097e8,0.0,0.0
"""0xb323495f7e41…",18926262,121,189,3.15263237e8,0.0,0.0
"""0xb323495f7e41…",18940531,93,258,3.73405563e8,646.0,0.0
"""0xc54d7acf14de…",18941243,38,104,2.60458612e8,0.0,0.0
"""0xc54d7acf14de…",18941407,66,103,2.08321558e8,0.0,0.0
